In [1]:

import codecs
import json
import configparser
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from sklearn import linear_model

%matplotlib inline

import dask.array as da
import dask.dataframe as dd
import h5py

from time import localtime, strftime

from sklearn.utils import shuffle

In [2]:
ID_FEATS = 3

In [3]:
# Load up config file (needs path; adapt env var if necessary); local imports
load_dotenv()

# load config file, set up paths, make project-specific imports
config_path = os.getenv('VISCONF')
if not config_path:
    # try default location, if not in environment
    default_path_to_config = '../Config/default.cfg'
    if os.path.isfile(default_path_to_config):
        config_path = default_path_to_config

assert config_path is not None, 'You need to specify the path to the config file via environment variable VISCONF.'        

config = configparser.ConfigParser()
with codecs.open(config_path, 'r', encoding='utf-8') as f:
    config.read_file(f)

corpora_base = config.get('DEFAULT', 'corpora_base')
preproc_path = config.get('DSGV-PATHS', 'preproc_path')
dsgv_home = config.get('DSGV-PATHS', 'dsgv_home')

sys.path.insert(0,dsgv_home + "/Utils")
from utils import icorpus_code, plot_labelled_bb, get_image_filename, query_by_id
from utils import plot_img_cropped, plot_img_ax, invert_dict, get_a_by_b, get_image_part
sys.path.insert(0,dsgv_home + "/WACs/WAC_Utils")
from wac_utils import create_word2den, is_relational, filter_refdf_by_filelist, filter_relational_expr
from wac_utils import filter_X_by_filelist, make_mask_matrix, make_X_id_index, train_this_word
from wac_utils import get_X_for_word

from data_utils import load_dfs

from apply_utils import apply_wac_set_matrix, logreg

#sys.path.append(dsgv_home + '/Preproc')

In [4]:
# Load up preprocessed DataFrames. Slow!
# These DataFrames are the result of pre-processing the original corpus data,
# as per dsg-vision/Preprocessing/preproc.py

df_names = ['refcoco_refdf', 'refcocoplus_refdf']
df = load_dfs(preproc_path, df_names)

In [5]:
with open(preproc_path + '/refcoco_splits.json', 'r') as f:
    rc_splits = json.load(f)

## RefCoco

In [6]:
# mscoco_bbdf_pattern = '/Volumes/BigData_SSD/Data/Computed/ExtractOut/vgg/mscoco_bbdf_vgg19-fc2/mscoco_bbdf_vgg19-fc2_%d.hdf5'
# model_path_prefix = '../TrainWACs/ModelsOut/01_refcoco_vgg'
mscoco_bbdf_pattern = '../../data/Models/ForBToma/mscoco_bbdf_rsn50-max/mscoco_bbdf_rsn50-max_%d.hdf5'
model_path_prefix = '../../data/Models/ForBToma/01_refcoco_rsn'

### Load Image Features

In [7]:
das = []
fhs = []
for n in range(1,8):
    f = h5py.File(mscoco_bbdf_pattern % (n), 'r')
    fhs.append(f)
    das.append(da.from_array(f['img_feats'], chunks=(1000, 4106)))

In [8]:
X = da.concatenate(das)

In [9]:
X.shape

(602408, 2058)

In [10]:
rc_all_test = rc_splits['testA'] + rc_splits['testB'] # rc_splits['val']  # rc_splits['testA'] + rc_splits['testB']
X_ts = filter_X_by_filelist(X, rc_all_test)
refdf_test = filter_refdf_by_filelist(df['refcoco_refdf'], rc_all_test)

In [11]:
# this is small enough to be fully in memory
X_ts = X_ts[:].compute()

In [12]:
# Meaning that I can already close the file handles
for fh in fhs:
    fh.close()

In [13]:
word2den_ts = create_word2den(refdf_test)
X_idx_ts = make_X_id_index(X_ts)
mask_matrix_ts = make_mask_matrix(X_ts, X_idx_ts, word2den_ts, word2den_ts.keys())

### Load WACs

In [14]:
def exp2indseq(w2i, exp):
    return [word2ind[w] for w in exp.split() if w in word2ind]

In [15]:
def imageid2rows(idx, ic, ii):
    '''return all regions that belong to an image, as indices into X (via idx)'''
    # or should this be a separate dictionary?
    return [v for k,v in idx.items() if k[0] == ic and k[1] == ii]

In [16]:
with h5py.File(model_path_prefix + '.hdf5', 'r') as f:
    wacs = f['wac_weights'][:]   # slice, to actually read into memory (as ndarray)

In [17]:
with codecs.open(model_path_prefix + '.json', 'r') as f:
    modelpars, wordlist = json.load(f)

### Apply all WACs to all regions

In [18]:
all_applied = apply_wac_set_matrix(X_ts[:, ID_FEATS:], wacs.T, net=logreg)

In [19]:
word2ind = {w[0]:n for n,w in enumerate(wordlist)}

In [20]:
def eval_corpus(rfdf, X, idx, w2i, all_applied):
    out = []
    for n, row in rfdf.iterrows():
        ic, ii, ri, refexp = row['i_corpus image_id region_id refexp'.split()]
        #if is_relational(refexp)
        all_regs = imageid2rows(idx, ic, ii)
        this_exp_seq = exp2indseq(w2i, refexp)
        all_regs_applied = np.prod(all_applied[all_regs][:, this_exp_seq], axis=1)
        regions_ranked = np.array(all_regs)[np.argsort(all_regs_applied)[::-1]]
        try:
            this_rank = np.where(X[regions_ranked][:, ID_FEATS-1] == ri)[0][0]
        except:
            print(ic, ii, ri, '\t region not in X?')
        out_of = len(all_regs)
        out.append((ic, ii, ri, refexp, is_relational(refexp), this_rank, out_of,
                    len(this_exp_seq) / len(refexp.split()) ))
    return pd.DataFrame(out, columns='i_corpus image_id region_id refexp is_rel rank n_obj perc_cov'.split())

In [21]:
outdf = eval_corpus(refdf_test, X_ts, X_idx_ts, word2ind, all_applied)

1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?


In [22]:
np.sum(outdf['rank'] == 0) / len(outdf)

0.556640625

In [23]:
np.sum(outdf['rank'] < 3) / len(outdf)

0.8722098214285714

In [24]:
def score_outdf_(subdf):
    print('accuracy @1: {:.2}'.format(np.sum(subdf['rank'] == 0) / len(subdf)))
    print('accuracy @3: {:.2}'.format(np.sum(subdf['rank'] < 3) / len(subdf)))
    print('mean reciprocal rank: {:.2}'.format(np.mean(1 / (subdf['rank'] + 1))))
    print('random baseline: {:.3}'.format(1 / np.mean(subdf['n_obj'])))


def score_outdf(outdf):
    print('** full')
    score_outdf_(outdf)
    print('** NR')
    score_outdf_(outdf[outdf['is_rel'] == False])
    print('** NR, cov > 0.5')
    score_outdf_(outdf[(outdf['is_rel'] == False) & (outdf['perc_cov'] >= 0.5)])

In [25]:
score_outdf(outdf)

** full
accuracy @1: 0.56
accuracy @3: 0.87
mean reciprocal rank: 0.72
random baseline: 0.0964
** NR
accuracy @1: 0.57
accuracy @3: 0.88
mean reciprocal rank: 0.73
random baseline: 0.0967
** NR, cov > 0.5
accuracy @1: 0.58
accuracy @3: 0.89
mean reciprocal rank: 0.74
random baseline: 0.0965


In [26]:
outdf_testA = eval_corpus(filter_refdf_by_filelist(df['refcoco_refdf'], rc_splits['testA']),
                          X_ts, X_idx_ts, word2ind, all_applied)

1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?


In [27]:
score_outdf(outdf_testA)

** full
accuracy @1: 0.57
accuracy @3: 0.87
mean reciprocal rank: 0.73
random baseline: 0.0796
** NR
accuracy @1: 0.58
accuracy @3: 0.88
mean reciprocal rank: 0.74
random baseline: 0.08
** NR, cov > 0.5
accuracy @1: 0.6
accuracy @3: 0.9
mean reciprocal rank: 0.75
random baseline: 0.0797


In [28]:
outdf_testB = eval_corpus(filter_refdf_by_filelist(df['refcoco_refdf'], rc_splits['testB']),
                          X_ts, X_idx_ts, word2ind, all_applied)

1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?


In [29]:
score_outdf(outdf_testB)

** full
accuracy @1: 0.54
accuracy @3: 0.87
mean reciprocal rank: 0.71
random baseline: 0.126
** NR
accuracy @1: 0.56
accuracy @3: 0.88
mean reciprocal rank: 0.72
random baseline: 0.127
** NR, cov > 0.5
accuracy @1: 0.57
accuracy @3: 0.89
mean reciprocal rank: 0.73
random baseline: 0.127


## RefCoco+

In [30]:
refdfp_test = filter_refdf_by_filelist(df['refcocoplus_refdf'], rc_splits['testA'] + rc_splits['testB'])
word2den_tsp = create_word2den(refdfp_test)

In [31]:
outdfp = eval_corpus(refdfp_test, X_ts, X_idx_ts, word2ind, all_applied)

1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region 

In [32]:
score_outdf(outdfp)

** full
accuracy @1: 0.44
accuracy @3: 0.83
mean reciprocal rank: 0.65
random baseline: 0.0965
** NR
accuracy @1: 0.46
accuracy @3: 0.83
mean reciprocal rank: 0.66
random baseline: 0.0963
** NR, cov > 0.5
accuracy @1: 0.47
accuracy @3: 0.85
mean reciprocal rank: 0.67
random baseline: 0.0951


In [33]:
score_outdf(eval_corpus(filter_refdf_by_filelist(df['refcocoplus_refdf'], rc_splits['testA']),
                        X_ts, X_idx_ts, word2ind, all_applied))

1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 534740 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 343009 489430 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 212675 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 195577 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61467 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 204792 61249 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 2152799 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?
1 3293 1721891 	 region not in X?
** full
accuracy @1: 0.49
accuracy @3: 0.84
mean reciprocal rank: 0.67
random baseline: 0.0797
** NR
accuracy @1: 0.5
accuracy @3: 0.84
mean reciprocal rank: 0.68
random ba

In [34]:
score_outdf(eval_corpus(filter_refdf_by_filelist(df['refcocoplus_refdf'], rc_splits['testB']),
                        X_ts, X_idx_ts, word2ind, all_applied))

1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 313360 1816504 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?
1 246356 1816289 	 region not in X?
** full
accuracy @1: 0.39
accuracy @3: 0.81
mean reciprocal rank: 0.62
random baseline: 0.128
** NR
accuracy @1: 0.41
accuracy @3: 0.82
mean reciprocal rank: 0.63
random baseline: 0.13
** NR, cov > 0.5
accuracy @1: 0.42
accuracy @3: 0.84
mean reciprocal rank: 0.64
random baseline: 0.129
